## Installation

To run Jupyter notebook locally:

```
python3 -m pip install "waymo_open_dataset_tf_2_6_0==1.4.3"
python3 -m pip install "notebook>=5.3" "ipywidgets>=7.5"
python3 -m pip install --upgrade jupyter_http_over_ws>=0.0.7 && \
jupyter serverextension enable --py jupyter_http_over_ws
jupyter notebook
```

In [ ]:
from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2
from waymo_open_dataset.protos import keypoint_pb2
from waymo_open_dataset import label_pb2
from waymo_open_dataset.utils import box_utils
from waymo_open_dataset.utils import keypoint_utils

In [ ]:
import tensorflow as tf

path = 'frame_with_keypoints.tfrecord'

dataset = tf.data.TFRecordDataset(path, compression_type='')
for data in dataset:
    frame = dataset_pb2.Frame()
    frame.ParseFromString(bytearray(data.numpy()))
    break

labels = keypoint_utils.group_object_labels(frame)
print(f'Loaded {len(labels)} objects')

In [ ]:
#@title Auixiliary imports and utils

import os
import tensorflow.compat.v1 as tf
import math
import numpy as np
from matplotlib import pylab as plt
import plotly.graph_objects as go
import itertools
import PIL.Image
import io
import dataclasses

tf.enable_eager_execution()

def _imdecode(buf):
  with io.BytesIO(buf) as fd:
    pil = PIL.Image.open(fd)
    return np.array(pil)

def _imshow(ax, image_np):
  ax.imshow(image_np)
  ax.axis('off')
  ax.set_autoscale_on(False)

def _draw_laser_points(fig, points, color='gray', size=3):
  fig.add_trace(go.Scatter3d(
    mode='markers',
    x=points[:, 0], y=points[:,1], z=points[:,2],
    marker=dict(
        color=color,
        size=size
    )
  ))

def _create_plotly_figure():
  fig = go.Figure()
  axis_settings = dict(
          showgrid=False,
          zeroline=False,
          showline=False,
          showbackground=False,
          showaxeslabels=False,
          showticklabels=False
      )
  fig.update_layout(
    width=600,
    height=600,
    showlegend=False,
    scene=dict(
      aspectmode = 'data',  # force xyz has same scale,
      xaxis=axis_settings,
      yaxis=axis_settings,
      zaxis=axis_settings,
    ),
  )
  return fig

In [ ]:
#@title Select object and camera
object_id = 'DQFLdFau_A8kTPOkDxfgJA'
camera_name = dataset_pb2.CameraName.Name.FRONT_RIGHT

camera_image_by_name = {i.name: i.image for i in frame.images}
obj = labels[object_id]
num_laser_points = len(obj.laser.keypoints.keypoint)
num_camera_points = len(obj.camera[camera_name].keypoints.keypoint)

print(f'Object {object_id} has')
print(f'{num_laser_points} laser keypoints (short name | location | is_occluded):')
for k in sorted(obj.laser.keypoints.keypoint, key=lambda k: k.type):
  m = k.keypoint_3d.location_m
  location_str = f'({m.x:.2f}, {m.y:.2f}, {m.z:.2f})'
  print(f'{keypoint_utils.point_name(k.type)}\t| {location_str:25} | {k.keypoint_3d.visibility.is_occluded}')
print(f'\na LaserKeypoint proto example:\n\n{obj.laser.keypoints.keypoint[0]}')

print(f'{num_camera_points} camera keypoints (short name |  location | is_occluded):')
for k in sorted(obj.camera[camera_name].keypoints.keypoint, key=lambda k: k.type):
  px = k.keypoint_2d.location_px
  location_str = f'({px.x:.0f}, {px.y:.0f})'
  print(f'{keypoint_utils.point_name(k.type)}\t| {location_str:13} | {k.keypoint_2d.visibility.is_occluded}')
print(f'\na CameraKeypoint proto example:\n\n{obj.camera[camera_name].keypoints.keypoint[0]}')

In [ ]:
#@title Show camera keypoints
image_np = _imdecode(camera_image_by_name[camera_name])
croped_image, cropped_camera_keypoints = keypoint_utils.crop_camera_keypoints(
    image_np,
    obj.camera[camera_name].keypoints.keypoint,
    obj.camera[camera_name].box,
    margin=0.3)
camera_wireframe = keypoint_utils.build_camera_wireframe(cropped_camera_keypoints)

keypoint_utils.OCCLUDED_BORDER_WIDTH = 3
_, ax = plt.subplots(frameon=False, figsize=(5, 7))
_imshow(ax, croped_image)
keypoint_utils.draw_camera_wireframe(ax, camera_wireframe)

In [ ]:
#@title Show laser keypoints

# Select laser points inside pedestrian's bounding box
(range_images, camera_projections, range_image_top_pose) = frame_utils.parse_range_image_and_camera_projection(frame)
points, cp_points = frame_utils.convert_range_image_to_point_cloud(
    frame,
    range_images,
    camera_projections,
    range_image_top_pose)
points_all = np.concatenate(points, axis=0)
box = box_utils.box_to_tensor(obj.laser.box)[tf.newaxis, :]
box_points = points_all[box_utils.is_within_box_3d(points_all, box)[:, 0]]
print(f'{box_points.shape[0]} laser points selected.')

# Visualize 3D scene
laser_wireframe = keypoint_utils.build_laser_wireframe(obj.laser.keypoints.keypoint)
fig = _create_plotly_figure()
keypoint_utils.draw_laser_wireframe(fig, laser_wireframe)
_draw_laser_points(fig, box_points)
fig.show()